In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame, points_from_xy
import folium
import json

In [28]:
vr = pd.read_csv("VaxRates.csv")
## Conversion to match GeoJSON data
vr["Zip"] = vr["Zip"].astype(int).astype(str)

In [29]:
## Should be 80 zip codes in Miami-Data county
display(len(vr['Zip'].value_counts()))
## one is missing from this data

79

In [30]:
## Load Florida zipcodes FeatureCollection
path = './fl_florida_zip_codes_geo.min.json'
with open(path) as json_file:
    FL_zip = json.load(json_file)

In [32]:
## Identify unique zipcodes to Miami county
## and keep only those from GeoJson file
zips = list(vr.Zip.unique())
FL_zip['features'] = [x for x in FL_zip['features'] if x['properties']['ZCTA5CE10'] in zips]

In [108]:
## Set zipcode order of dataframe
## match zipcode order of GeoJson
geo_ord = []
for idx in range(len(FL_zip['features'][:])):
    geo_ord.append(FL_zip['features'][idx]['properties']['ZCTA5CE10'])
sortidx = dict(zip(geo_ord, range(len(geo_ord))))
vr['geoOrder'] = vr['Zip'].map(sortidx)
vr = vr.set_index('geoOrder')

In [110]:
## Add custom hover data
## Prep custom text
tooltip_text = []
for idx in range(len(vr)):
    tooltip_text.append("Zip: "+vr['Zip'][idx]+', Vaxxed='+str(vr['percent_Adults_AtLeastOneDose'][idx])+'%, Poverty='+
                       str(vr['Percent_Poverty'][idx])+'%')
# tooltip_text

## Append tooltip column
## to GeoJSON file
for idx in range(len(tooltip_text)):
    FL_zip['features'][idx]['properties']['tooltip1'] = tooltip_text[idx]

In [112]:
## Create Folium Map
m = folium.Map(location=[25.72, -80.1918], zoom_start=9)

vaxxed = folium.Choropleth(highlight=True,
    geo_data=FL_zip,
    name='choropleth',
    data=vr,
    columns=['Zip', 'percent_Adults_AtLeastOneDose'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='BuGn',
    fill_opacity=0.8,
    line_opacity=0.3,
    legend_name='Percent of Adults with at least One Dose'
).add_to(m)

folium.LayerControl().add_to(m)
vaxxed.geojson.add_child(
    folium.features.GeoJsonTooltip(['tooltip1'], labels=False)
)

display(m)